In [20]:
from oauth2client import file
from oauth2client import tools
from oauth2client import client
from pprint import pprint
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import httplib2
import json
import collections
import requests
from datetime import datetime, date
from pytz import timezone

In [2]:
api_name = 'analytics'
api_version = 'v3'
profile_id = '186677014'

In [3]:
storage = file.Storage(api_name + '.dat')
credentials = storage.get()

In [4]:
http = credentials.authorize(http=httplib2.Http())
service = build(api_name, api_version, http=http)

In [5]:
config = json.load(open('analytics.dat'))
client_id = config['client_id']
client_secret = config['client_secret']
refresh_token = config['refresh_token']
token = config['access_token']

In [15]:
def update_token(client_id, client_secret, refresh_token):
    """Обновление токена для запросов API. Возвращает токен."""
    url_token = 'https://accounts.google.com/o/oauth2/token'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token,
        'grant_type': 'refresh_token',
    }
    r = requests.post(url=url_token, data=params)
    print(f'Токен выдан до {datetime.today() + timedelta(hours=1)}')
    return r.json()['access_token']

In [16]:
update_token(client_id, client_secret, refresh_token)

Токен выдан до 2020-12-26 15:31:59.830135


'ya29.a0AfH6SMD7-EbbnhtHEQthLiCU5MoTcO11tMQAhoaHpG1mX-SMwvK56OxCaTMtWCuuYfUgtqpBxzBWunyesUdcKDHA1dllfgTFK1LZpA1j9uJFNkHTZTCI-p9p81VQBZZz-h0DjV4MjDjLkngRPjDzbjf7Imz8KVWxxbRWUb1C-hk'

In [17]:
today = datetime.strftime(date.today(), '%Y-%m-%d')

In [40]:
data = service.data().ga().get(
    ids='ga:' + profile_id,
    start_date='2018-02-12',
    end_date=today,
    metrics='ga:visits',
    dimensions='ga:dateHourMinute',
    sort='-ga:visits',
    filters='ga:medium==organic',
    start_index='1',
    max_results='25')\
.execute()
data

{'kind': 'analytics#gaData',
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:186677014&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2020-12-26&start-index=1&max-results=25',
 'query': {'start-date': '2018-02-12',
  'end-date': '2020-12-26',
  'ids': 'ga:186677014',
  'dimensions': 'ga:dateHourMinute',
  'metrics': ['ga:visits'],
  'sort': ['-ga:visits'],
  'filters': 'ga:medium==organic',
  'start-index': 1,
  'max-results': 25},
 'itemsPerPage': 25,
 'totalResults': 665,
 'selfLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:186677014&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-date=2018-02-12&end-date=2020-12-26&start-index=1&max-results=25',
 'nextLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:186677014&dimensions=ga:dateHourMinute&metrics=ga:visits&sort=-ga:visits&filters=ga:medium%3D%3Dorganic&start-dat

In [10]:
result = data['rows']
sorted(result)

[['201812152258', '1'],
 ['201812160013', '1'],
 ['201812160024', '1'],
 ['201812220506', '1'],
 ['201812221307', '1'],
 ['201812231033', '1'],
 ['201812231908', '1'],
 ['201812232025', '1'],
 ['201812232047', '1'],
 ['201812241038', '1'],
 ['201812242147', '1'],
 ['201812250757', '1'],
 ['201812251443', '1'],
 ['201812260704', '1'],
 ['201812271643', '1'],
 ['201812281115', '1'],
 ['201812290527', '1'],
 ['201812300604', '1'],
 ['201901061918', '1'],
 ['201901092018', '1'],
 ['201901111449', '1'],
 ['201901112051', '1'],
 ['201909291650', '2'],
 ['202006071043', '3'],
 ['202006071055', '2']]

In [11]:
count = 0
for visit in result:
   count += int(visit[1])

print(count)

29


In [12]:
def report(start_index, max_results):
    try:
        data = service.data().ga().get(
        ids='ga:' + '186677014',
        start_date='2018-02-12',
        end_date=datetime.strftime(date.today(), '%Y-%m-%d'),
        metrics='ga:visits',
        dimensions='ga:dateHourMinute',
        sort='-ga:visits',
        filters='ga:medium==organic',
        start_index=start_index,
        max_results=max_results).execute()
        return data['rows']
    except KeyError:
        'Выгрузка окончена, больше данных нет'

In [13]:
start_index = 1
max_results = 500
results = []
while True:
    print('Начинаю выгрузку для start_index = {}'.format(start_index))
    data = report(start_index, max_results)
    
#     pprint(result)
    if data == None:
        break

   
    results += data
    start_index += max_results


Начинаю выгрузку для start_index = 1
Начинаю выгрузку для start_index = 501
Начинаю выгрузку для start_index = 1001


In [37]:
pprint(results[:])

[['202006071043', '3'],
 ['201909291650', '2'],
 ['202006071055', '2'],
 ['201812152258', '1'],
 ['201812160013', '1'],
 ['201812160024', '1'],
 ['201812220506', '1'],
 ['201812221307', '1'],
 ['201812231033', '1'],
 ['201812231908', '1'],
 ['201812232025', '1'],
 ['201812232047', '1'],
 ['201812241038', '1'],
 ['201812242147', '1'],
 ['201812250757', '1'],
 ['201812251443', '1'],
 ['201812260704', '1'],
 ['201812271643', '1'],
 ['201812281115', '1'],
 ['201812290527', '1'],
 ['201812300604', '1'],
 ['201901061918', '1'],
 ['201901092018', '1'],
 ['201901111449', '1'],
 ['201901112051', '1'],
 ['201901120029', '1'],
 ['201901121203', '1'],
 ['201901122117', '1'],
 ['201901141724', '1'],
 ['201901142136', '1'],
 ['201901151147', '1'],
 ['201901151608', '1'],
 ['201901181554', '1'],
 ['201901181602', '1'],
 ['201901192105', '1'],
 ['201901201822', '1'],
 ['201901202328', '1'],
 ['201901211925', '1'],
 ['201901221122', '1'],
 ['201901231500', '1'],
 ['201901231604', '1'],
 ['201901231642'

In [26]:
c = collections.Counter()

In [36]:
count = 0
for elem in results:
    dig = int(elem[1])
    count += dig
print(count)

669


In [41]:
3+4-3

4

In [42]:
669-4

665